In [21]:
#Step 1: Install Required Libraries
!pip install transformers datasets torch pandas scikit-learn accelerate -U
!pip install numpy
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments

# Disable WandB
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [23]:
# Load the dataset
dataset = load_dataset("SetFit/sst5")

# Convert the dataset to pandas DataFrames
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

# Check the column names
print(train_df.columns)

# Split the train_df into training and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

# Reset index
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

Repo card metadata block was not found. Setting CardData to empty.


Index(['text', 'label', 'label_text'], dtype='object')


In [24]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_df['text'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True)

train_labels = train_df['label'].tolist()
val_labels = val_df['label'].tolist()
test_labels = test_df['label'].tolist()


In [25]:
import torch

class SST5Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SST5Dataset(train_encodings, train_labels)
val_dataset = SST5Dataset(val_encodings, val_labels)
test_dataset = SST5Dataset(test_encodings, test_labels)


In [26]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the model
model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=5)

# Define the metrics
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    evaluation_strategy="epoch",     
    save_strategy="epoch",  # Make sure the evaluation and save strategy match
    save_total_limit=2,              
    num_train_epochs=10,             
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=32,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    disable_tqdm=False,
    report_to=[]  # Disable wandb
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,             
    compute_metrics=compute_metrics,      
)

# Train the model
trainer.train()


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.474600,1.314771,0.422222,0.337340,0.293320,0.422222
2,1.135600,1.172504,0.480702,0.413644,0.459823,0.480702
3,1.041600,1.185500,0.474854,0.472933,0.476938,0.474854
4,0.942300,1.282487,0.476023,0.466245,0.470821,0.476023
5,0.718000,1.367177,0.506433,0.502044,0.504463,0.506433
6,0.427800,1.764314,0.479532,0.474055,0.476105,0.479532
7,0.301900,1.938149,0.493567,0.489302,0.488491,0.493567
8,0.311900,2.241622,0.481871,0.484432,0.488620,0.481871
9,0.222800,2.663580,0.507602,0.507822,0.514464,0.507602
10,0.188600,2.911621,0.495906,0.495896,0.496708,0.495906


/Users/jigarpatel/.pyenv/versions/eleven/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=4810, training_loss=0.6960889729603412, metrics={'train_runtime': 2275.3841, 'train_samples_per_second': 33.792, 'train_steps_per_second': 2.114, 'total_flos': 3714313424924520.0, 'train_loss': 0.6960889729603412, 'epoch': 10.0})

In [27]:
# Evaluate on the test set
results = trainer.evaluate(test_dataset)
print(results)

{'eval_loss': 2.586169481277466, 'eval_accuracy': 0.5104072398190045, 'eval_f1': 0.5119425333216943, 'eval_precision': 0.5290126488627277, 'eval_recall': 0.5104072398190045, 'eval_runtime': 8.781, 'eval_samples_per_second': 251.679, 'eval_steps_per_second': 7.972, 'epoch': 10.0}


In [28]:

#from huggingface_hub import notebook_login

#notebook_login()

In [29]:
#trainer.push_to_hub()